# Cut My TESS Light Curve

## Introduction

TESS does a search of the postage-stamp, two-minute cadence data. For every signal it identifies with its Transit Planet Search (TPS) module, it creates something called a Threshold Crossing Event (TCE).  TCEs are periodic signals that exceed a nominal signal-to-noise requirement. Some are consistent with transiting planets, others are eclipsing binaries, and others are more consistent with variable stars or noise in the data. 

The Data Validation (DV) module of the pipeline produces a set of products that can help validate the quality of a TCE. The DV products include a time series file of the flattened light curve that was searched and relevant statistics for each signal (dvt.fits), DV reports that consists of a few diagnostic plots and relevant statistics (dvs.pdf for individual signals, dvr.pdf for all signals found in the TIC object), and an xml file (dvr.xml) that contains the results of the planet transit fit.  We will be exploring a dvt.fits file in this tutorial.

This tutorial will refer to a couple TESS-related terms that we define here.

* Sector = TESS observed the sky in regions of 24x96 degrees along the southern, then northern, ecliptic hemispheres.  Each of these regions is referred to as a "sector", starting with Sector 1.
* HDU = Header Data Unit.  A FITS file is made up of HDUs that contain data and metadata relating to the file. The first HDU is called the primary HDU, and anything that follows is considered an "extension", e.g., "the first FITS extension", "the second FITS extension", etc.
* BJD = Barycentric Julian Date, the Julian Date that has been corrected for differences in the Earth's position with respect to the Solar System center of mass.
* TBJD = TESS Barycentric Julian Date, the timestamp measured in BJD, but offset by 2457000.0.  I.e., TBJD = BJD - 2457000.0
* Cadence = The interval between flux measurements, nominally ~2 minutes for the target pixel files, and ~30 minutes for the full frame images.
* TCE = Threshold Crossing Event, periodic signals found by the TESS pipeline that exceed a nominal signal-to-noise ratio.

## Obtaining The DVT Series File

We will read the data validation file of our star (HAT-P-36 as an example) first from Sector 22 using the local copy we downloaded from the [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html) so that we can get started with understanding the file contents without reviewing how to automatically search for and retrieve TESS files, we won't show how to search and retrieve TESS DVT files in this tutorial. Consult other TESS tutorials [in the TESS Archive Handbook](https://outerspace.stsci.edu/display/TESS/6.0+-+Data+Search+Tutorials) for much more information on how to search for TESS data using MAST services.  If you haven't downloaded yet you can search for your target and download the 120 (or 20 for more recent observations) second products by clicking on the save button next to the light curve icon. Please download the latest version if you have more than 120 (or 20) second data for the same sector because the pipeline is re-run on the data when it is updated.

But let's import all the required modules first.

In [ ]:
# import all the necessary model to run this code first
%matplotlib inline
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Get the name of the dvt file name from the user:

Please enter the file name of your Data Validation Timeseries (DVT) file. This is the file with the suffix <i>"_dvt.fits"</i> among the data files you downloaded from the MAST. If it is in another directory than this code then enter its name together with its full or relative path.

In [ ]:
dvt_file = input("Please enter your filename\n")

## Understanding The DVT File Structure

The DVT FITS file consits of a primary HDU with metadata stored in the header, and one FITS extension HDU per TCE found in the lightcurve of the specified TIC ID.  These extensions contain the detrended flux time series phased to the orbital period of the signal, stored as a binary FITS table.  The last extension HDU always contains some additional statistics about the search, also stored in a binary FITS table.  Let's examine the structure of the FITS file using the astropy.fits `info` function, which shows the FITS file format in more detail.

In [ ]:
fits.info(dvt_file)

In this case, HAT-P-36 has one TCE's (Threshold Crossing Event) identified, and the "statistics" extension in the last HDU, as expected. TCE number can be more than one. But let's examine the only TCE extension in more detail using the astropy.fits `getdata` function and see what columns are available.

In [ ]:
fits.getdata(dvt_file, ext=1).columns

In addition to the timestamps in TBJD format, there is a column containing the times phased to the orbital period of the signal, and there are several columns of fluxes.  LC_INIT is the "unwhitened" fluxes, LC_WHITE are the "whitened" fluxes.  The MODEL_INIT and MODEL_WHITE fluxes are the corresponding model fluxes based on the best fit to the signal. 

## Plotting The Timeseries Fluxes.

Let's open the FITS file and extract some metadata from the headers, and also store some of the columns from the first TCE signal for use later when we plot the results.

We are going the need the orbital period ($P_{\rm orb}$), the reference mid-transit time ($T_0$) and the duration of the full transit {$T_{\rm full}$). You can make use of the the stellar radius scaled to the semi-major axis ($\frac{R_{\star}}{a}$), the radius ratio ($\frac{R_p}{R_{\star}}$), the impact parameter ($b$), and the orbital inclination ($i$) all derived from the preliminary model by the TESS pipeline to compute the transit duration too. But we need the transit duration only to cut our light curve. We are going to update this model anyway in the exofast and measure the mid-transit time from that model. For its calculation please see the other tutorials. However in order to cut our light curve, we will need only these three parameters that are listed in the columns of the header for the TCE signal, as you will see below. The parameter that are not required are commented out for you to see what other options you have!

In [ ]:
with fits.open(dvt_file, mode="readonly") as hdulist:
    
    # You can extract stellar parameters from the primary header (hdulist[0]). 
    # We'll get the effective temperature, surface gravity,
    # and TESS magnitude as an example (we are not going to use them!)
    #star_teff = hdulist[0].header['TEFF']
    #star_logg = hdulist[0].header['LOGG']
    #star_tmag = hdulist[0].header['TESSMAG']
    
    # Extract some of the fit parameters for the first TCE (hdulist[1]). 
    # These are stored in the FITS header of the first
    # extension.
    Porb = hdulist[1].header['TPERIOD']
    T_14 = hdulist[1].header['TDUR']/24.0
    T0 = hdulist[1].header['TEPOCH'] + 2457000.0
    #a_Rs = hdulist[1].header['DRRATIO']
    #Rp_Rs = hdulist[1].header['RADRATIO']
    #b = hdulist[1].header['IMPACT']
    
    # Now the light curve:
    lc_dvt_2min = hdulist[1].data

In [ ]:
%matplotlib inline
plt.errorbar(lc_dvt_2min['TIME']+2457000., lc_dvt_2min['LC_DETREND']+1.00, yerr=lc_dvt_2min['LC_INIT_ERR'],\
            fmt='.', c='b',  label='LC-DETREND Flux')

plt.legend(loc='best')
plt.show()

If you'd like you can check and verify the reference elements by plotting the light curve with respect to the orbital phase.

In [ ]:
%matplotlib inline
plt.errorbar(lc_dvt_2min['PHASE'], lc_dvt_2min['LC_DETREND']+1.00, yerr=lc_dvt_2min['LC_INIT_ERR'],\
            fmt='.', c='b',  label='LC-DETREND Flux')

plt.legend(loc='best')
plt.show()

# Individual Transits

Now we want to look at individual transits. For this we should compute a parameter called the epoch of observation as well call it in astronomy, which has a trivial formula. It actually gives you how many orbital periods that your planet covered, the integer part of which gives you the number of orbital periods and the decimal part gives you the fraction of how many periods have been taken at any given time after the reference mid-transit time ($T_0$).

$$ E = \frac{(T - T_0)}{P_{\rm orb}} $$

$T$ being the time that you would like to know the corresponding epoch. The calculation is simple in python. But let's continue with a Pandas data frame for the individual light curves.

In [ ]:
lcfull = pd.DataFrame({'time':lc_dvt_2min['TIME']+2457000,
                      'flux':lc_dvt_2min['LC_DETREND']+1.00,
                       'flux_err':lc_dvt_2min['LC_INIT_ERR']})
lcfull['epoch'] = (lcfull['time'] - T0) / Porb
lcfull['epoch']

Obviously, the first transit is around 0.00, and the last transit is around 20.0 in our sample. So the integer part of the epoch tells us to which orbital cycle each point in time on our light curve corresponds to. However, we would not like to take an entire orbital cycle but rather an individual transit. The decimal part will be the key in determination of the data points we should cut. At this point, the transit duration comes into play. We will decide how many hours of the observation we should cut from the entire light curve to form an individual transit light curve based on this value too. We have already taken this but it is in days. The conventional unit for the full transit duration is in minutes so that you can compare your results with that from any transiting exoplanet catalogue such as [ETD](http://var2.astro.cz/ETD/) or [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PS) for your planet. So let's multiply this number with 1440, the number of minutes in a days. 

We need to convert this value to the fraction of an orbital period, so that we know how much of a cycle is covered by the transit. Then we are going to take 1.5 full transit durations before and after the transit center so that our individual light curves will cover 3 full transit transit durations, 1 of which is for the transit and 1 for each of the out-of-transit level. Considering the uncertainties on measurements, this will be a safe bet.

In [ ]:
# T14 as a fraction of an orbital period
T_14_frac = T_14 / Porb
print("Full transit duration is {:.2f} minutes".format(T_14*1440))
print("Full transit duration covers {:.4f} of an entire orbit".format(T_14_frac))

## Cutting The Individual Transits

Now that we are done with the calculations, we are going to cut the individual transits which will cover 3 full transit durations and save them to different files with the following code:

In [ ]:
for i,Ee in enumerate(lcfull['epoch']):
    Ee += 1.5*T_14_frac
    E = int(Ee)
    if (Ee <= (E + 3*T_14_frac)):
        fname = "tess_lc_{:d}".format(E)
        file2wr = open(fname,"a")
        str2wr = "{:.6f}\t{:.8f}\t{:.8f}\n".format(lcfull['time'][i],lcfull['flux'][i],lcfull['flux_err'][i])
        file2wr.write(str2wr)

## Visual Inspection of the Individual Transits

Now let's look at each light curve and make sure it covers the transit as we like it. Some of the transits might not be present or full due to gaps in the data. We should note and ignore these files, and then delete them to prevent future confusion.

In [ ]:
import glob
from matplotlib import pyplot as plt
fnames = glob.glob("./tess_lc_*", recursive=False)
for i,fn in enumerate(fnames):
    plt.figure(i+1)
    lc = pd.read_csv(fn, delimiter="\t",header=None)
    print("Light Curve: ", fn)
    plt.errorbar(lc[0],lc[1],lc[2])
    plt.show()
    lc.to_csv(fn, sep="\t",header=["BJD-TDB","flux","flux_err"])

# Final

You can now analyze your light curves in AIJ, detrend them there if you'd like and model them in Exofast